In [ ]:
import intake
import hvplot.xarray

In [ ]:
gui = intake.gui
gui

In [ ]:
gui.add('https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml')

### Ocean

In [ ]:
Select `ocean(sea_surface_height)`

In [ ]:
sea_surface_ds = gui.sources[0].to_dask()
sea_surface_ds

### Atmosphere

In [ ]:
gui

In [ ]:
Select `atmosphere(gmet_v1)`

In [ ]:
gmet_v1_ds = gui.sources[0].to_dask()
gmet_v1_ds

### Hydro

In [ ]:
gui

In [ ]:
Select `hydro(cgiar_pet)`

In [ ]:
cgiar_pet_ds = gui.sources[0].to_dask()

In [ ]:
cgiar_pet_ds